In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1"    
import tensorflow as tf
from pathlib import Path
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import time
import cv2
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

%matplotlib inline

In [2]:
PATH_TO_FROZEN_GRAPH = 'freeze/frozen_inference_graph.pb'
PATH_TO_LABELS = 'train_files/label_map.pbtxt'

In [3]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [4]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

In [5]:
keys = ['num_detections', 'detection_boxes', 'detection_scores', 'detection_classes']
tensor_dict = {key: key+':0' for key in keys}
image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
def run_inference_for_single_image(image_path, graph, sess):
    image = Image.open(image_path)
    image_np = np.array(image)
    image_np_expanded = np.expand_dims(image, axis=0)
    start_time = time.time()
    # Run inference
    output_dict = sess.run(
        tensor_dict,
        feed_dict={image_tensor: image_np_expanded})
    # all outputs are float32 numpy arrays, so convert types as appropriate
    output_dict['num_detections'] = int(output_dict['num_detections'][0])
    output_dict['detection_classes'] = output_dict[
      'detection_classes'][0].astype(np.uint8)
    output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
    output_dict['detection_scores'] = output_dict['detection_scores'][0]
    print(time.time() - start_time)
    return output_dict

In [6]:
sess = tf.Session(graph=detection_graph)

In [7]:
output_dict = run_inference_for_single_image(
    '../analysis1/dataset/images/IMG_20191102_103759.jpg', detection_graph, sess)

2.1130008697509766


In [8]:
output_dict = run_inference_for_single_image(
    '../analysis1/dataset/images/IMG_20191102_103808.jpg', detection_graph, sess)

0.14948034286499023


In [9]:
output_dict = run_inference_for_single_image(
    '../analysis1/dataset/images/IMG_20191102_103821.jpg', detection_graph, sess)

0.1578507423400879


In [10]:
detect_idx = np.argmax(output_dict['detection_scores'])
detection = output_dict['detection_boxes'][detect_idx]
detection

array([0.10217437, 0.01671228, 0.86999905, 0.64430296], dtype=float32)

In [11]:
vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks'),
      use_normalized_coordinates=True,
      line_thickness=10)
plt.imshow(image_np)

NameError: name 'image_np' is not defined